In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
url = "https://rating.hh.ru/history/rating2023/summary?tab=giant" 

xpaths = [
    '//*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[1]',
    '//*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[2]',
    '//*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[3]',
    '//*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[4]'
]

nametags = ['Крупнейшие', 'Крупные', 'Средние', 'Небольшие']

service = Service(executable_path=r'chromedriver-win64\chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()
driver.get(url)

dataframes = {} 
for name, xpath in zip(nametags, xpaths): 

    element = driver.find_element(By.XPATH, xpath)
    ActionChains(driver).move_to_element(element).click().perform()  # Hover before clicking (if necessary)
    
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "_176_F2jG6H"))
    )

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find("table", class_="_176_F2jG6H")

    data = []
    rows = table.find("tbody").find_all("tr")
    for row in rows:
        columns = row.find_all("td")
        
        row_data = [col.get_text(strip=True) for col in columns]

        data.append(row_data)

    headers = [header.get_text(strip=True) for header in table.find_all("th")]
    df = pd.DataFrame(data, columns=headers)
    dataframes[name] = df

In [55]:
for name, df in dataframes.items():
    df.to_csv(rf'parsed\{name}_HH_rating_2023.csv')


In [56]:
dataframes['Крупнейшие']

,Место,Название компании,Регион,Отрасль,Итоговый балл
0,1,Альфа-Банк,Москва,Банки,"130,04"
1,2,Яндекс,Москва,IT и интернет,"119,37"
2,3,ВТБ,Москва,Банки,"118,12"
3,4,Tele2,Москва,"Телекоммуникации, связь","112,88"
4,5,"""Аэрофлот""",Москва,"Транспорт, логистика, автодилеры","112,04"
...,...,...,...,...,...
124,125,Командор,Красноярский край,Розничные сети,"89,36"
125,126,Читай-город,Москва,Розничные сети,"89,11"
126,127,Дальневосточная распределительная сетевая комп...,Амурская область,"Энергетика, добыча и переработка сырья","86,95"
127,128,РЕШЕТНЁВ,Красноярский край,"Промышленное производство (непищевое), торгово...","85,91"


In [43]:
dataframes['Крупные']

,Место,Название компании,Отрасль
0,1,2ГИС,IT и интернет
1,2,Издательство Эксмо,"Маркетинг, реклама, издательский бизнес"
2,3,Банк Точка,Банки
3,4,«Фоксфорд»,Наука и образование
4,5,ВЭБ.РФ,Государственная служба и организации
...,...,...,...
350,351,ФГАУ РОСЖИЛКОМПЛЕКС,Государственная служба и организации
351,352,Бахетле,Розничные сети
352,353,Уралэлектротяжмаш (УЭТМ),"Промышленное производство (непищевое), торгово..."
353,354,Такском,IT и интернет


In [44]:
dataframes['Средние']

,Место,Название компании,Отрасль
0,1,Иви,Медиа
1,2,СберМаркетинг,"Маркетинг, реклама, издательский бизнес"
2,3,Okko,Медиа
3,4,Агентство инноваций Москвы,Государственная служба и организации
4,5,Москвариум,Развлекательные сети
...,...,...,...
474,475,Аэропорт Оренбург,"Транспорт, логистика, автодилеры"
475,476,Кувалда.ру,Розничные сети
476,477,Международный аэропорт Нижний Новгород,"Транспорт, логистика, автодилеры"
477,478,Цветочный Ряд (сеть цветочных супермаркетов),Розничные сети


In [45]:
dataframes['Небольшие']

,Место,Название компании,Отрасль
0,1,ЦЕНТР РАЗВИТИЯ «СРЕДА ВОЗМОЖНОСТЕЙ»,Государственная служба и организации
1,2,BLACK STAR,Медиа
2,3,Sports.ru,Медиа
3,4,Афиша,Медиа
4,5,VERTEX,Сервисные и инжиниринговые компании
...,...,...,...
529,530,Полис,Фармацевтика и медицина
530,531,Траверс,Розничные сети
531,532,Интегрикс,IT и интернет
532,533,НПН,"Оптовые продажи, дилеры, дистрибьюторы"


In [38]:


service = Service(executable_path=r'chromedriver-win64\chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)


xpaths = [
    '//*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[1]',
    '//*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[2]',
    '//*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[3]',
    '//*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[4]'
]

# Click each element by XPath
for xpath in xpaths:
    try:
        element = driver.find_element(By.XPATH, xpath)
        ActionChains(driver).move_to_element(element).click().perform()  # Hover before clicking (if necessary)
        print(f"Clicked element at: {xpath}")
    except Exception as e:
        print(f"Could not click element at {xpath}: {e}")


Clicked element at: //*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[1]
Clicked element at: //*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[2]
Clicked element at: //*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[3]
Clicked element at: //*[@id="root"]/div/div/div/div[1]/div/div[7]/div[1]/div[4]
